## Computing Gaia luminosities

### Read in Kepler seismic values

In [ ]:
seis = pd.read_csv('/home/nielsemb/work/the_big_solar_like_oscillator_list.csv')

### Read in Gaia database values

In [ ]:
gaia = pickle.load(open( "/home/nielsemb/work/search_result_g.p", "rb" ) ).GDR2.to_pandas()
idxg = gaia['source_id'] == 0
gaia[idxg] = np.nan

In [ ]:
def GbandBolometricCorrection(teff, teff_0=5777):

    BC_G = np.zeros_like(teff, dtype = float)
    sigma_BC_G = np.zeros_like(teff, dtype = float)
    
    idx0 = teff > 4000
    # Coefficients for BC_G(Teff) and sigma(BC_G)
    p0 = np.array([[ 6.000e-02,  2.634e-02],
                   [ 6.731e-05,  2.438e-05],
                   [-6.647e-08, -1.129e-09],
                   [ 2.859e-11, -6.722e-12],
                   [-7.197e-15,  1.635e-15]])
    
    idx1 = teff <=4000
    p1 = np.array([[ 1.749e+00,-2.487e+00],
                   [ 1.977e-03,-1.876e-03],
                   [ 3.737e-07, 2.128e-07],
                   [-8.966e-11, 3.807e-10],
                   [-4.183e-14, 6.570e-14]])
    
    BC_G[idx0] = np.polyval(p0[::-1,0], teff[idx0] - teff_0)
    BC_G[idx1] = np.polyval(p1[::-1,0], teff[idx1] - teff_0)
    
    sigma_BC_G[idx0] = np.polyval(p0[::-1,1], teff[idx0] - teff_0)
    sigma_BC_G[idx1] = np.polyval(p1[::-1,1], teff[idx1] - teff_0)

    return BC_G, sigma_BC_G

def luminosity(teff, m_G, plx, A_G=0, Mbol_0 = 4.75):
    
    idx = plx <=0
    plx[idx] = np.nan
    
    BC_G, sigma_BC_G = GbandBolometricCorrection(teff, teff_0=5772)
     
    logL = 4.0 - 2.0*np.log10(plx) - 0.4*(m_G - A_G + BC_G - Mbol_0)
    
    return 10**logL

In [ ]:
BC = GbandBolometricCorrection(df['teff'], teff_0=5777)
fig, ax = subplots(figsize = (16,9))
ax.scatter(df['teff'],BC[0])
ax.scatter(df['teff'],BC[0]-BC[1], s = 3)
ax.scatter(df['teff'],BC[0]+BC[1], s = 3)
ax.invert_xaxis()

In [ ]:
offset = 0.029 #mas
L = luminosity(seis['teff'], gaia['phot_g_mean_mag'], gaia['parallax']+offset, gaia['a_g_val'])

## Estimating $\nu_{max}$

In [ ]:
def seismicParameters(teff, L, numax_0 = 3050, teff_0 = 5777):
    """ Estimated seismic parameters for the star
    
    Based on Teff and L
    
    """

    numax = L**-0.9 * (teff/teff_0)**4.62 * numax_0
    
    return numax

## Compare Gaia and computed luminosities with TRILEGAL

In [ ]:
tril = pd.read_csv('/home/nielsemb/work/repos/PSM128/P5_selection/platosim_STEP01.dat', sep = ' ')
idx1 = 10**tril.logTe < 8000
idx2 = (tril.mu0 < (np.log10(150000)-1)*5) # Limit on distance modulus
idx3 = (tril.mratio == 0) # mass ratio?? limit 
idx4 = (tril.label < 9) # ?? 
idx5 = (tril.Gc < 2) # ??
idxCut = idx1 & idx2 & idx3 & idx4 & idx5

fig, ax = subplots(figsize = (16,9))
ax.scatter(10**tril['logTe'][idxCut], tril['logL'][idxCut], alpha = 0.75)
ax.scatter(df['teff'], np.log10(gf['lum_val']), alpha = 0.2)
ax.scatter(df['teff'], np.log10(L), alpha = 0.2)
ax.invert_xaxis()

## Compare $\nu_{max}$ using different luminosity sources

In [ ]:
gaia_numax = seismicParameters(df['teff'], gf['lum_val'])
comp_numax = seismicParameters(df['teff'], L)
tril_numax = seismicParameters(10**tril['logTe'].values, 10**tril['logL'].values, )



fig, ax = subplots(figsize = (16,16))
ax.scatter(tril['logL'], np.log10(tril_numax), alpha = 0.2, c = tril['logTe'], s = 2)
ax.scatter(np.log10(gf['lum_val']), np.log10(df['numax']), alpha = 0.1, c = 'C1', s = 10)
ax.scatter(np.log10(L), np.log10(df['numax']), alpha = 0.1, c = 'C3', s = 10)
ax.set_ylim(0,4)
ax.set_xlim(-1,3)
ax.invert_yaxis()

fig, ax = subplots(figsize = (16,9))
ax.scatter(np.log10(df['numax']), np.log(g_numax0) - np.log(df['numax']), alpha = 0.25)
ax.scatter(np.log10(df['numax']), np.log(g_numax1) - np.log(df['numax']), alpha = 0.25)
#ax.set_yscale('log')
ax.set_ylim(-2,2)
#ylim(-1,1)
#ax.axhline(0, color = 'C1')
#ax.axhline(0+np.log10(3), color = 'C2')
#ax.axhline(0-np.log10(3), color = 'C2')